## This demo presents the implementation for the following stories:
- RSPY-326 - The staging process for cadip in cluster mode
- RSPY-475 - The deletion of the s3 files from the s3 bucket beside the deletion of the info from the catalog

In [ ]:
import requests
import os
import pprint
# Init environment before running a demo notebook.
from resources.utils import *

pp = pprint.PrettyPrinter(indent=2, width=80, sort_dicts=False, compact=True)
session = requests.Session()
user = os.environ["JUPYTERHUB_USER"] if cluster_mode else os.environ["RSPY_HOST_USER"]
auxip_client, cadip_client, stac_client, staging_client = init_demo(owner_id = user)
if os.getenv("RSPY_LOCAL_MODE") == "1":
    href = "http://rs-server-cadip:8000"    
    href_staging = "http://rs-server-staging:8000"
else:
    href = os.environ["RSPY_WEBSITE"]
    href_staging = "https://rsserverstaging.dev-rspy.esa-copernicus.eu"
    session.cookies.set ("session", os.environ["RSPY_OAUTH2_COOKIE"])

cadip_collection_id = "cadip"
adgs_collection_id = "adgs"


In [ ]:
# Create a test collection 
collection = create_test_collection()

### Check the added collection with the rs-server-catalog. This collection should be empty.

In [ ]:
# Check the catalog for agrosu_my_test_collection
result = session.get(f"{stac_client.href_catalog}/catalog/collections/{user}:{TEST_COLLECTION}/items")
catalog_collection = result.json()
assert catalog_collection.get("type") == "FeatureCollection"
assert catalog_collection.get("context").get("returned") == 0
print(f"No items found in the '{TEST_COLLECTION}' collection")

### Getting all the sessions from cadip_s1A collection found in the configuration of CADIP station

In [ ]:
result = session.get(f"{href}/cadip/collections/{cadip_collection_id}/items")
items_collection = result.json()
assert items_collection.get("type") == "FeatureCollection"
assert len(items_collection.get("features")) > 0
for item in items_collection.get("features"):
    print(f"Session {item.get('id')} has {len(item.get('assets'))} assets with datetime {item.get('properties').get('datetime')}")    

### Creating a staging body to start the staging process

In [ ]:
TIMEOUT = 10
staging_body = {
    "version": "0.2.0",
    "id": "staging",
    "title": {
        "en": "Staging"
    },
    "description": {
        "en": "A process that takes an external STAC ItemCollection, asynchronously download its assets into the RS catalog bucket and creates the corresponding STAC items in the RS catalog."
    },
    "jobControlOptions": [
        "async-execute"
    ],
    "keywords": [
        "stac",
        "staging"
    ],
    "links": [
        {
            "type": "text/html",
            "rel": "about",
            "title": "documentation",
            "href": "https://home.rs-python.eu/rs-documentation/rs-server/docs/doc/users/functionalities/#staging",
            "hreflang": "en-US"
        }
    ],
    "inputs": {
        "collection": {
            "title": "Target collection",
            "description": "The target collection identifier in the RS catalog",
            "id": TEST_COLLECTION,
            "schema": {
                "type": "string"
            },
            "minOccurs": 1,
            "maxOccurs": 1
        },
        "items": items_collection,
        "provider": "cadip"
    },
    "outputs": {
        "result": {
            "title": "Output STAC items",
            "id": "some_output_id",
            "description": "The staged STAC ItemCollection",
            "schema": "false",
            "minOccurs": 1,
            "maxOccurs": 1
        }
    }
}

import requests
post_response = session.post(f"{href_staging}/processes/staging/execution", 
                              json=staging_body,                              
                              timeout = TIMEOUT,)

resp = json.loads(post_response.content)
pprint.PrettyPrinter(indent=4).pprint(resp)

job_id = resp["status"]["started"]
print(f"\nThe job_id = {job_id}\n")

import time
timeout = 120
while timeout > 0:
    post_response = requests.get(f"{href_staging}/jobs/{job_id}",
                              **apikey_headers,
                              timeout = TIMEOUT,)
    try:
        resp = json.loads(post_response.content)
        pprint.PrettyPrinter(indent=4).pprint(resp)
        print("\n")
        if resp["status"] == "FINISHED":
            print("Job COMPLETED")
            break
            
        if resp["status"] == "FAILED":
            print("Job FAILED")
            break
    except (    
            json.JSONDecodeError,
        ):        
        continue
    time.sleep(2)
    timeout -= 2

### Check the catalog for agrosu_my_test_collection. Four items should be present now

In [ ]:
# Check the catalog for agrosu_my_test_collection
result = session.get(f"{stac_client.href_catalog}/catalog/collections/{TEST_COLLECTION}/items")
catalog_collection = result.json()
assert catalog_collection.get("type") == "FeatureCollection"
assert len(catalog_collection.get("features")) == 5
for item in catalog_collection.get("features"):
    print(f"Item {item.get('id')} has {len(item.get('assets'))} assets")    

### Delete one item from the collection

In [ ]:
item_to_delete = "S1A_20220715090550123456"
result = session.delete(f"{stac_client.href_catalog}/catalog/collections/{TEST_COLLECTION}/items/{item_to_delete}")
assert result.json()["deleted item"] == "S1A_20220715090550123456"
pp.pprint(result.json())

### Delete the whole collection

In [ ]:
result = session.delete(f"{stac_client.href_catalog}/catalog/collections/{TEST_COLLECTION}")
assert result.json()["deleted collection"] == TEST_COLLECTION
pp.pprint(result.json())